In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.2 MB/s eta 0:00:00


# Service 1

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
import optuna

# Load and preprocess data
df = pd.read_csv("Service1.csv")

features = ['latency_ms', 'memory_allocated', 'cpu_allocated', 'cpu_usage_pct', 'memory_usage_pct']
targets = ['cpu_usage_pct', 'memory_usage_pct']

# Normalize features
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

X = df[features].values
y = df[['cpu_usage_pct', 'memory_usage_pct']].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Optuna objective
def objective(trial):
    max_depth = trial.suggest_int("max_depth", 3, 30)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    model_cpu = DecisionTreeRegressor(max_depth=max_depth,
                                      min_samples_split=min_samples_split,
                                      min_samples_leaf=min_samples_leaf)
    model_mem = DecisionTreeRegressor(max_depth=max_depth,
                                      min_samples_split=min_samples_split,
                                      min_samples_leaf=min_samples_leaf)

    model_cpu.fit(X_train, y_train[:, 0])
    model_mem.fit(X_train, y_train[:, 1])

    pred_cpu = model_cpu.predict(X_test)
    pred_mem = model_mem.predict(X_test)

    mae_cpu = mean_absolute_error(y_test[:, 0], pred_cpu)
    mae_mem = mean_absolute_error(y_test[:, 1], pred_mem)

    return mae_cpu + mae_mem

# Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# Train final model with best hyperparameters
best_params = study.best_params
final_model_cpu = DecisionTreeRegressor(**best_params)
final_model_mem = DecisionTreeRegressor(**best_params)

final_model_cpu.fit(X_train, y_train[:, 0])
final_model_mem.fit(X_train, y_train[:, 1])

# Predict
cpu_pred = final_model_cpu.predict(X_test)
mem_pred = final_model_mem.predict(X_test)

# Accuracy
mae_cpu_final = mean_absolute_error(y_test[:, 0], cpu_pred)
mae_mem_final = mean_absolute_error(y_test[:, 1], mem_pred)

print("Best Hyperparameters:", best_params)
print(f"MAE CPU Usage Prediction: {mae_cpu_final:.4f}")
print(f"MAE Memory Usage Prediction: {mae_mem_final:.4f}")


[I 2025-05-05 10:21:28,607] A new study created in memory with name: no-name-40170bad-b1ac-4ccc-ba78-0788cb14f51f
[I 2025-05-05 10:21:28,736] Trial 0 finished with value: 0.0009023148384740895 and parameters: {'max_depth': 15, 'min_samples_split': 11, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.0009023148384740895.
[I 2025-05-05 10:21:28,861] Trial 1 finished with value: 0.0010650507208829602 and parameters: {'max_depth': 9, 'min_samples_split': 19, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.0009023148384740895.
[I 2025-05-05 10:21:28,923] Trial 2 finished with value: 0.01958763949131417 and parameters: {'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.0009023148384740895.
[I 2025-05-05 10:21:29,065] Trial 3 finished with value: 0.0007920992714602487 and parameters: {'max_depth': 26, 'min_samples_split': 15, 'min_samples_leaf': 2}. Best is trial 3 with value: 0.0007920992714602487.
[I 2025-05-05 10:21:29,399] Trial 4 fin

Best Hyperparameters: {'max_depth': 23, 'min_samples_split': 3, 'min_samples_leaf': 1}
MAE CPU Usage Prediction: 0.0001
MAE Memory Usage Prediction: 0.0001
